Test check

In [8]:
import findspark
findspark.init('/home/nasty280700/spark-3.2.4-bin-hadoop3.2')
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import count

In [2]:
!pip install pandas
!pip install pyarrow
!pip install fastparquet

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
print("<<---***--- START ---***--->>")

spark = (SparkSession
 .builder
 .appName('pyspark_example')
 .enableHiveSupport()
 .getOrCreate())
 
rows = [
(1,"Tom", "Green", "Washington"),(2,"Alice", "Black", "Washington"),
(3,"Jimmy", "Kimmel", "Texas"), (4,"Bob", "White", "Texas"),
(5,"Kenny", "West", "Atlanta"), (6,"Kate", "Tompson", "Texas")
]

print(type(rows))
print(rows[1])
print(type(rows[1]))

schema = "customer_id BIGINT, fname STRING, lname STRING, state STRING"
customersDF = spark.createDataFrame(rows, schema)
customersDF.show()

customersDF.createOrReplaceTempView("tmp_clients")

clientsDF = spark.sql("""
select distinct state as states from tmp_clients
""")

clientsDF.show()

print("<<---***--- END ---***--->>")
spark.stop()

<<---***--- START ---***--->>


23/08/27 12:26:40 WARN Utils: Your hostname, nasty280700-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/08/27 12:26:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/27 12:26:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<class 'list'>
(2, 'Alice', 'Black', 'Washington')
<class 'tuple'>


+-----------+-----+-------+----------+
|customer_id|fname|  lname|     state|
+-----------+-----+-------+----------+
|          1|  Tom|  Green|Washington|
|          2|Alice|  Black|Washington|
|          3|Jimmy| Kimmel|     Texas|
|          4|  Bob|  White|     Texas|
|          5|Kenny|   West|   Atlanta|
|          6| Kate|Tompson|     Texas|
+-----------+-----+-------+----------+



+----------+
|    states|
+----------+
|     Texas|
|Washington|
|   Atlanta|
+----------+

<<---***--- END ---***--->>


In [12]:
import pandas as pd

spark = (SparkSession
         .builder
         .appName("Task1")
         .enableHiveSupport()
         .getOrCreate())

data = [
    Row(row_id=1, discipline="athletics", season="summer"),
    Row(row_id=2, discipline="swimming", season="summer"),
    Row(row_id=3, discipline="artistic gymnastics", season="summer"),
    Row(row_id=4, discipline="cycling road", season="summer"),
    Row(row_id=5, discipline="volleyball", season="summer"),
    Row(row_id=6, discipline="bobsleigh", season="winter"),
    Row(row_id=7, discipline="ice hockey", season="winter"),
    Row(row_id=8, discipline="biathlon", season="winter"),
    Row(row_id=9, discipline="ice skating", season="winter"),
    Row(row_id=10, discipline="snowboarding", season="winter")
]


df = spark.createDataFrame(data)

df.show()
df.coalesce(1).write.option("sep", "\t").csv("task1_olymp_folder.csv", header=True, mode="overwrite")
df.toPandas().to_csv("task1_olymp.csv", index = False, sep="\t")

spark.stop()

+------+-------------------+------+
|row_id|         discipline|season|
+------+-------------------+------+
|     1|          athletics|summer|
|     2|           swimming|summer|
|     3|artistic gymnastics|summer|
|     4|       cycling road|summer|
|     5|         volleyball|summer|
|     6|          bobsleigh|winter|
|     7|         ice hockey|winter|
|     8|           biathlon|winter|
|     9|        ice skating|winter|
|    10|       snowboarding|winter|
+------+-------------------+------+



In [5]:
spark = (SparkSession
         .builder
         .appName("Task2")
         .enableHiveSupport()
         .getOrCreate())

df = spark.read.option("delimiter", ";").csv("Athletes.csv", header=True)

df.createOrReplaceTempView("athletes")

df.show()

df_count = spark.sql("""
select Discipline, 
    count(Name) as athletes_count 
from athletes 
group by Discipline
""")
df_count.show()

df_count.write.parquet("discipline_count_folder.parquet", mode="overwrite")
df_count.toPandas().to_parquet("discipline_count.parquet", index = False)

spark.stop()

+--------------------+--------------------+-------------------+
|                Name|                 NOC|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [6]:
import pyarrow.parquet as pq

table = pq.read_table("discipline_count.parquet")
#table = pq.read_table("/home/nasty280700/spark-3.2.4-bin-hadoop3.2/discipline_count.parquet")

df = table.to_pandas()

df.head()

,Discipline,athletes_count
0,Tennis,178
1,Boxing,270
2,Marathon Swimming,49
3,Golf,115
4,Rowing,496


In [9]:
from pyspark.sql.functions import initcap

spark = (SparkSession
         .builder
         .appName("Task3")
         .enableHiveSupport()
         .getOrCreate())

df_file = spark.read.option("delimiter", ";").csv("Athletes.csv", header=True)

df_count = df_file.groupBy("Discipline").agg(count("Name").alias("athletes_count"))
df_count.show()

data = [
    Row(row_id=1, discipline="athletics", season="summer"),
    Row(row_id=2, discipline="swimming", season="summer"),
    Row(row_id=3, discipline="artistic gymnastics", season="summer"),
    Row(row_id=4, discipline="cycling road", season="summer"),
    Row(row_id=5, discipline="volleyball", season="summer"),
    Row(row_id=6, discipline="bobsleigh", season="winter"),
    Row(row_id=7, discipline="ice hockey", season="winter"),
    Row(row_id=8, discipline="biathlon", season="winter"),
    Row(row_id=9, discipline="ice skating", season="winter"),
    Row(row_id=10, discipline="snowboarding", season="winter")
]

df_task1 = spark.createDataFrame(data)
df_task1 = df_task1.withColumnRenamed("discipline", "Discipline")
df_task1 = df_task1.withColumn("Discipline", initcap(df_task1["Discipline"]))


res_df = df_count.join(df_task1, "Discipline", 'inner')
res_df.show()


res_df.write.parquet("1_join_2_folder.parquet", mode="overwrite")
res_df.toPandas().to_parquet("1_join_2.parquet", index = False)

spark.stop()

+--------------------+--------------+
|          Discipline|athletes_count|
+--------------------+--------------+
|              Tennis|           178|
|              Boxing|           270|
|   Marathon Swimming|            49|
|                Golf|           115|
|              Rowing|           496|
|   Baseball/Softball|           220|
|                Judo|           373|
|             Sailing|           336|
|            Swimming|           743|
|Cycling BMX Frees...|            19|
|          Basketball|           280|
|            Handball|           343|
| Rhythmic Gymnastics|            95|
|              Karate|            77|
|           Triathlon|           106|
|           Badminton|           164|
|        Canoe Sprint|           236|
|           Athletics|          2068|
|       Cycling Track|           208|
|    Beach Volleyball|            90|
+--------------------+--------------+
only showing top 20 rows



+-------------------+--------------+------+------+
|         Discipline|athletes_count|row_id|season|
+-------------------+--------------+------+------+
|          Athletics|          2068|     1|summer|
|           Swimming|           743|     2|summer|
|Artistic Gymnastics|           187|     3|summer|
|       Cycling Road|           190|     4|summer|
|         Volleyball|           274|     5|summer|
+-------------------+--------------+------+------+



In [10]:
#table = pq.read_table("1_join_2.parquet")
table = pq.read_table("/home/nasty280700/spark-3.2.4-bin-hadoop3.2/1_join_2.parquet")

df = table.to_pandas()

df.head()

,Discipline,athletes_count,row_id,season
0,Athletics,2068,1,summer
1,Swimming,743,2,summer
2,Artistic Gymnastics,187,3,summer
3,Cycling Road,190,4,summer
4,Volleyball,274,5,summer
